## Применение нейронных сетей в анализе звука

In [1]:
import librosa
import pandas as pd
import numpy as np
import scipy
import sklearn
import os
import csv

Рассмотрим аудио файлы в наборе данных:

In [2]:
wav1_dir, _, wav1_files = next(os.walk('audio/svm/wav1'))
wav2_dir, _, wav2_files = next(os.walk('audio/svm/wav2'))
wav3_dir, _, wav3_files = next(os.walk('audio/svm/wav3'))
wav4_dir, _, wav4_files = next(os.walk('audio/svm/wav4'))
print(f"wav1 files: {len(wav1_files)}\nwav2 files: {len(wav2_files)}\nwav3 files: {len(wav3_files)}\nwav4 files: {len(wav4_files)}")

wav1 files: 10
wav2 files: 10
wav3 files: 10
wav4 files: 10


Из всех аудиофайлов в наборе данных с помощью библиотеки librosa - librosa.feature, метода append( ) и метода extend( ) проводим:
<li> Извлечение из Мел-кепстральных коэффициентов - средние значения и стандартные отклонения (по 13 значений); </li>
<li> Извлечение из Спектрального центроида - среднее значение, стандартное отклонение и skew (наклон); </li>
<li> Извлечение из Спектрального спада - среднее значение и стандартное отклонение; </li>

In [3]:
def extract_features(directory, file):
    name = f'{directory}/{file}'
    y, sr = librosa.load(name, mono=True, duration=5)
    
    features = [] 
    features.append(file)                                                                           # filename
    features.extend([np.mean(e) for e in librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)])              # mfcc_mean<0..13>
    features.extend([np.std(e) for e in librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)])               # mfcc_std<0..13>
    features.append(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr).T, axis = 0)[0])          # cent_mean
    features.append(np.std(librosa.feature.spectral_centroid(y=y,sr=sr).T, axis = 0)[0])            # cent_std
    features.append(scipy.stats.skew(librosa.feature.spectral_centroid(y=y,sr=sr).T, axis = 0)[0])  # cent_skew
    features.append(np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr).T, axis = 0)[0])           # rolloff_mean
    features.append(np.std(librosa.feature.spectral_rolloff(y=y, sr=sr).T, axis = 0)[0])            # rolloff_std
    features.append(directory.split('/')[-1])
    
    return features

Извлечение и сохранение значений в формате CSV файла:

In [4]:
buffer = []
buffer_size = 5000
buffer_counter = 0

header = ['filename']
header.extend([f'mfcc_mean{i}' for i in range(1, 14)])
header.extend([f'mfcc_std{i}' for i in range(1, 14)])
header.extend(['cent_mean', 'cent_std', 'cent_skew', 'rolloff_mean', 'rolloff_std', 'label'])

with open('dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(header)
    for directory, files in [(wav1_dir, wav1_files), (wav2_dir, wav2_files), (wav3_dir, wav3_files), (wav4_dir, wav4_files)]:
        for file in files:
            features = extract_features(directory, file)
            if buffer_counter + 1 == buffer_size:
                buffer.append(features)
                writer.writerows(buffer)
                print(f"- [{directory.split('/')[-1]}] Write {len(buffer)} rows")
                buffer = []
                buffer_counter = 0
            else:
                buffer.append(features)
                buffer_counter += 1
        if buffer:
            writer.writerows(buffer)
            print(f"- [{directory.split('/')[-1]}] Write {len(buffer)} rows") 
        print(f"- [{directory.split('/')[-1]}] Writing complete")
        buffer = []
        buffer_counter = 0

- [wav1] Write 10 rows
- [wav1] Writing complete
- [wav2] Write 10 rows
- [wav2] Writing complete
- [wav3] Write 10 rows
- [wav3] Writing complete
- [wav4] Write 10 rows
- [wav4] Writing complete


Загрузка CSV файла

In [5]:
data = pd.read_csv('dataset.csv')
data[0:5]

,filename,mfcc_mean1,mfcc_mean2,mfcc_mean3,mfcc_mean4,mfcc_mean5,mfcc_mean6,mfcc_mean7,mfcc_mean8,mfcc_mean9,...,mfcc_std10,mfcc_std11,mfcc_std12,mfcc_std13,cent_mean,cent_std,cent_skew,rolloff_mean,rolloff_std,label
0,e0346.wav,-499.87265,119.811070,-69.236660,16.326145,-2.472070,-14.168089,-16.978210,-9.742417,-8.744191,...,10.882130,13.849910,11.041399,9.977660,2076.460751,843.737212,1.217495,3606.213379,1645.860409,wav1
1,e0347.wav,-523.26440,121.582800,-85.195260,28.111979,-2.020563,-4.897916,-20.038010,-11.397475,-8.751575,...,14.846037,10.153871,11.159391,8.089673,2100.568407,810.919912,1.137475,3235.313924,1281.917788,wav1
2,e0348.wav,-535.78906,122.248955,-68.690620,13.530365,-5.120312,-15.022529,-19.569761,-5.859893,-8.050721,...,15.039412,11.968068,10.047459,9.637810,2103.405813,909.587344,1.141859,3578.798421,1672.138327,wav1
3,e0349.wav,-565.35504,105.944530,-58.878956,16.648294,-0.114853,-9.569576,-10.512609,-3.712021,-4.109455,...,14.083085,11.421207,8.447350,7.460106,2280.545331,738.086737,0.639644,4140.206909,1596.796477,wav1
4,e0350.wav,-539.70496,107.440480,-63.355556,21.695900,4.090648,-9.275818,-17.341760,-4.672110,-10.452682,...,14.248458,12.401202,10.857938,9.210695,2214.048516,896.925732,1.017873,3745.730591,1635.356403,wav1


In [6]:
data.shape

(40, 33)

<li> столбец filename —  номер и название файла. </li> 
<li> столбцы mfcc_mean{i} и mfcc_std{i} - средние значения и стандартные отклонения (по 13 значений) из Мел-кепстральных коэффициентов. </li> 
<li> столбцы cent_mean, cent_std, cent_skew - среднее значение, стандартное отклонение и skew (наклон) из Спектрального центроида. </li> 
<li> столбцы rolloff_mean, rolloff_std - среднее значение и стандартное отклонение из Спектрального спада. </li> 
<li> столбец label — метка. </li> 

Целевой атрибут в классификации является категориальной переменной в колонке label - это метка (правильный ответ). Учитывая набор обучающих точек данных вместе с целевыми метками, классификация определяет метку класса для немаркированного тестового случая.

Посмотрим на подсчет значений в колонке label.

In [7]:
data['label'].value_counts()

wav2    10
wav3    10
wav1    10
wav4    10
Name: label, dtype: int64

Пусть y - зависимая (целевая) переменная (колонка label)

In [8]:
y = data['label'].values
y[0:5]

array(['wav1', 'wav1', 'wav1', 'wav1', 'wav1'], dtype=object)

Посмотрим на название колонок в наборе данных:

In [9]:
data.columns

Index(['filename', 'mfcc_mean1', 'mfcc_mean2', 'mfcc_mean3', 'mfcc_mean4',
       'mfcc_mean5', 'mfcc_mean6', 'mfcc_mean7', 'mfcc_mean8', 'mfcc_mean9',
       'mfcc_mean10', 'mfcc_mean11', 'mfcc_mean12', 'mfcc_mean13', 'mfcc_std1',
       'mfcc_std2', 'mfcc_std3', 'mfcc_std4', 'mfcc_std5', 'mfcc_std6',
       'mfcc_std7', 'mfcc_std8', 'mfcc_std9', 'mfcc_std10', 'mfcc_std11',
       'mfcc_std12', 'mfcc_std13', 'cent_mean', 'cent_std', 'cent_skew',
       'rolloff_mean', 'rolloff_std', 'label'],
      dtype='object')

Пусть X - независимые переменные (значимые характеристики аудиоданных)

In [10]:
X = data[['mfcc_mean1', 'mfcc_mean2', 'mfcc_mean3', 'mfcc_mean4',
       'mfcc_mean5', 'mfcc_mean6', 'mfcc_mean7', 'mfcc_mean8', 'mfcc_mean9',
       'mfcc_mean10', 'mfcc_mean11', 'mfcc_mean12', 'mfcc_mean13', 'mfcc_std1',
       'mfcc_std2', 'mfcc_std3', 'mfcc_std4', 'mfcc_std5', 'mfcc_std6',
       'mfcc_std7', 'mfcc_std8', 'mfcc_std9', 'mfcc_std10', 'mfcc_std11',
       'mfcc_std12', 'mfcc_std13', 'cent_mean', 'cent_std', 'cent_skew',
       'rolloff_mean', 'rolloff_std']]
X[0:5]

,mfcc_mean1,mfcc_mean2,mfcc_mean3,mfcc_mean4,mfcc_mean5,mfcc_mean6,mfcc_mean7,mfcc_mean8,mfcc_mean9,mfcc_mean10,...,mfcc_std9,mfcc_std10,mfcc_std11,mfcc_std12,mfcc_std13,cent_mean,cent_std,cent_skew,rolloff_mean,rolloff_std
0,-499.87265,119.811070,-69.236660,16.326145,-2.472070,-14.168089,-16.978210,-9.742417,-8.744191,-7.401200,...,12.331656,10.882130,13.849910,11.041399,9.977660,2076.460751,843.737212,1.217495,3606.213379,1645.860409
1,-523.26440,121.582800,-85.195260,28.111979,-2.020563,-4.897916,-20.038010,-11.397475,-8.751575,-11.714219,...,12.370551,14.846037,10.153871,11.159391,8.089673,2100.568407,810.919912,1.137475,3235.313924,1281.917788
2,-535.78906,122.248955,-68.690620,13.530365,-5.120312,-15.022529,-19.569761,-5.859893,-8.050721,-12.130071,...,14.160710,15.039412,11.968068,10.047459,9.637810,2103.405813,909.587344,1.141859,3578.798421,1672.138327
3,-565.35504,105.944530,-58.878956,16.648294,-0.114853,-9.569576,-10.512609,-3.712021,-4.109455,-7.870796,...,13.689024,14.083085,11.421207,8.447350,7.460106,2280.545331,738.086737,0.639644,4140.206909,1596.796477
4,-539.70496,107.440480,-63.355556,21.695900,4.090648,-9.275818,-17.341760,-4.672110,-10.452682,-11.815496,...,14.379725,14.248458,12.401202,10.857938,9.210695,2214.048516,896.925732,1.017873,3745.730591,1635.356403


Если в наборе данных есть некоторые выбросы или значения переменных находятся в разных диапазонах, необходимо исправить их. 
Стандартизация данных дает нулевое среднее значение и единичную дисперсию.

In [11]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:2]

array([[-0.64893425,  1.2795014 , -1.57273071, -0.35729162, -0.23215378,
        -1.25110953, -0.68617175, -0.93690099, -0.60070567, -0.42668987,
         0.6141391 , -0.02948268,  0.67089388,  0.27245456,  0.214985  ,
         1.72052111,  1.27601498,  1.35390756,  0.82092133,  0.34114654,
         1.41954698, -0.74358627, -0.90696898,  1.11006989,  0.37937571,
         0.75930047, -0.13799005, -0.93069505,  0.29828881, -0.64763177,
        -0.84494485],
       [-0.88761035,  1.37504486, -2.06036174,  0.779931  , -0.16781674,
        -0.40796915, -0.89923847, -1.11052176, -0.60200605, -1.00500938,
         0.23792842, -0.42863978,  0.38786207, -1.41682663, -0.17293583,
         0.90234038,  2.21920962,  1.08012773,  0.91290785,  1.07666855,
         2.24527402, -0.72776881,  1.04480163, -0.57731988,  0.46994716,
        -0.52624082, -0.05123389, -1.06674109,  0.18218141, -1.36518321,
        -1.85866996]])

Разделение набора данных на тренировочные и тестовые наборы (train/test split):

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

print ('Train set:', X_train.shape, y_train.shape)    
print ('Test set:', X_test.shape, y_test.shape)

Train set: (32, 31) (32,)
Test set: (8, 31) (8,)


Обучим модель SVM (Support Vector Machines):

In [13]:
from sklearn import svm

clf = svm.SVC(kernel='rbf')   # функция ядра - RBF (радиальная базисная функция)
clf.fit(X_train, y_train)     # Обучение модели на тренировочном наборе
yhat = clf.predict(X_test)    # для прогнозирования новых значений:
yhat[0:10]

array(['wav4', 'wav2', 'wav2', 'wav4', 'wav3', 'wav4', 'wav2', 'wav3'],
      dtype=object)

SVM  воспринимается как один из самых мощных алгоритмов обучения сложных нелинейных функции, 
очень широко используется как в промышленности, так и в академических кругах. 
И по сравнению с логистической регрессией и нейронными сетями, 
Support Vector Machine иногда дает более чистый, а иногда более мощный способ обучения сложным нелинейным функциям.

Сравнение значений:

In [14]:
print("Prediction:", yhat[0:20])
print("Real Value:", y_test[0:20])

Prediction: ['wav4' 'wav2' 'wav2' 'wav4' 'wav3' 'wav4' 'wav2' 'wav3']
Real Value: ['wav4' 'wav2' 'wav2' 'wav4' 'wav3' 'wav4' 'wav2' 'wav3']


Эффективность модели:

In [15]:
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat) )

Train set Accuracy:  1.0
Test set Accuracy:  1.0


Accuracy — оценка классификации точности - это функция, которая вычисляет точность подмножества, 
долю правильных ответов алгоритма. 
По сути, он вычисляет, насколько точно совпадают фактические и прогнозируемые метки в тестовом наборе.

In [16]:
from sklearn.metrics import classification_report, confusion_matrix

print('CONFUSION_MATRIX :\n')
print(confusion_matrix(y_test, yhat))
print('\n')

print('REPORT :\n')
print(classification_report(y_test, yhat))

CONFUSION_MATRIX :

[[3 0 0]
 [0 2 0]
 [0 0 3]]


REPORT :

              precision    recall  f1-score   support

        wav2       1.00      1.00      1.00         3
        wav3       1.00      1.00      1.00         2
        wav4       1.00      1.00      1.00         3

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



F1 score — Confusion matrix показывает корректные и неправильные прогнозы по сравнению с фактическими метками. Каждая строка матрицы неточностей показывает метки Actual/True в тестовом наборе, а столбцы показывают прогнозируемые метки по классификатору. Мы можем интерпретировать эти числа как количество истинных положительных, ложных отрицательных, истинных отрицательных и ложных срабатываний.  Матрица неточностей показывает способность модели правильно предсказать или разделить классы.

Precision — это мера точности при условии, что метка класса была предсказана.

Recall — измеряет способность модели обнаруживать выборки, при условии, что метка класса была предсказана.

Support — количество верно предсказанных ответов в данном классе.